# Import libraries 

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import numpy as np

from time import time

# Simple example with MNIST dataset

## Download the MNIST dataset

In [2]:
dataset =  datasets.MNIST('../data', download=True)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


## Define simple convolution network

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

## Train the classification model

In [4]:
# "Map function": acquires a corresponding Cloud TPU core, creates a tensor on it,
# and prints its core
batch_size=128
epochs=10
lr=1
gamma=0.7
log_interval=10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_dataset =  datasets.MNIST('../data', train=True, download=True,
                  transform=transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,), (0.3081,))]))

test_dataset = datasets.MNIST('../data', train=False, transform=transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,), (0.3081,))]))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                           num_workers=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, 
                                          num_workers=4, shuffle=False)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

print("Start with %s"%device)

start_trn_t, device_times, epoch_times = time(), [], []
for epoch in range(1, epochs + 1):
  model.train()
  start_epoch_t = time()
  for batch_idx, (data, target) in enumerate(train_loader):
      start_it_t = time()
      data, target = data.to(device), target.to(device)
      optimizer.zero_grad()
      output = model(data)
      loss = F.nll_loss(output, target)
      loss.backward()
      optimizer.step()
      device_times.append(time()-start_it_t)
  print("[Epoch #%d] [Train] Total time: %.3f(s/epoch)\t Avg. time: %.3f(s/iter)"%(epoch, time()-start_epoch_t, device_times[-1]))
 
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
      for data, target in test_loader:
          data, target = data.to(device), target.to(device)
          output = model(data)
          test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
          pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
          correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)

  print('[Epoch #%d] [Test] Average loss: %.4f, Accuracy: %d/%d (%.1f%%)' %(
      epoch, test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))
  scheduler.step()
  epoch_times.append(time()-start_epoch_t)
print("Train time: %.3f, Avg. time: %.3f(s/iter), std:%.3f, Avg. time: %.3f(s/epoch), std:%.3f"\
        %(time()-start_trn_t, np.mean(device_times), np.std(device_times),  np.mean(epoch_times), np.std(epoch_times)))


Start with cuda
[Epoch #1] [Train] Total time: 4.809(s/epoch)	 Avg. time: 0.005(s/iter)
[Epoch #1] [Test] Average loss: 0.0693, Accuracy: 9781/10000 (97.8%)
[Epoch #2] [Train] Total time: 4.556(s/epoch)	 Avg. time: 0.003(s/iter)
[Epoch #2] [Test] Average loss: 0.0356, Accuracy: 9871/10000 (98.7%)
[Epoch #3] [Train] Total time: 4.654(s/epoch)	 Avg. time: 0.004(s/iter)
[Epoch #3] [Test] Average loss: 0.0317, Accuracy: 9897/10000 (99.0%)
[Epoch #4] [Train] Total time: 4.667(s/epoch)	 Avg. time: 0.004(s/iter)
[Epoch #4] [Test] Average loss: 0.0279, Accuracy: 9900/10000 (99.0%)
[Epoch #5] [Train] Total time: 4.658(s/epoch)	 Avg. time: 0.004(s/iter)
[Epoch #5] [Test] Average loss: 0.0287, Accuracy: 9900/10000 (99.0%)
[Epoch #6] [Train] Total time: 4.590(s/epoch)	 Avg. time: 0.003(s/iter)
[Epoch #6] [Test] Average loss: 0.0271, Accuracy: 9906/10000 (99.1%)
[Epoch #7] [Train] Total time: 4.614(s/epoch)	 Avg. time: 0.004(s/iter)
[Epoch #7] [Test] Average loss: 0.0255, Accuracy: 9911/10000 (99.1